# Duboko učenje - laboratorijska vježba - generativni modeli - Generative adversarial networks (GAN)

Primarna namjena GAN-a je isto generiranje novih i uvjerljivih uzoraka, no princip rada je malo drugačiji od prethodna dva modela. GAN ne procjenjuje direktno parametre $p(\mathbf x)$ ili bilo koje druge distribucije, premda se njegovo treniranje može interpretirati kao estimacija $p(\mathbf x)$. Najvjerojatnije zahvaljujući tom drugačijem pristupu, GAN-ovi često generiraju vizualno najbolje uzorke u usporedbi sa VAE ili drugim generativnim mrežama.

GAN se sastoji od dvije zasebne mreže 

1. Generator (G) koji ima zadatak generirati uvjerljive uzorke
2. Diskriminator (D) koji ima zadatak prepoznati radi li se o pravom uzorku (iz skupa za treniranje) ili lažnom uzorku koji je generirao G

<center>
<div class="fig figcenter fighighlight">
  <img src="https://dlunizg.github.io/assets/lab4/GAN.svg" width="30%">
</div>
</center>

Te dvije mreže su protivnici (Adversaries), imaju dijametralno suprotstavljene ciljeve te se pokušavaju nadmudriti. To nadmetanje ih tjera da budu sve bolji u postizanju svog cilja i da se fokusiraju na sve bitne detalje ulaznih podataka. Očekivano, njihovo nadmetanje trebalo bi dovesti do toga da generator generira savršene uzorke koje diskriminator ne može razlikovati od uzoraka iz skupa za treniranje. Da bi generator postigao takav uspjeh nužno je da i diskriminator bude maksimalno dobar u svom zadatku.

Generator na svojem izlazu generira uzorke za neki slučajni ulazni vektor koji prati neku distribuciju. Ta slučajnost na ulazu omogućuje generatoru da uvijek generira nove uzorke. Pri tome nema nekih posebnih ograničenja na arhitekturu generatora, no poželjno je da se može trenirati backpropagation algoritmom. 

<center>
<div class="fig figcenter fighighlight">
  <img src="https://dlunizg.github.io/assets/lab4/G.svg" width="30%">
</div>
</center>

Diskriminator na svome izlazu treba estimirati pripadnost razredu stvarnih ili lažnih uzoraka za svaki ulazni vektor. Za razliku od generatora, ovdje je moguće koristiti učenje pod nadzorom jer se za svaki uzorak zna da li je došao iz skupa za treniranje ili od generatora. Radi jednostavnosti možemo izlaz diskriminatora ograničiti u rasponu $[0,1]$ i interpretirati kao vjerojatnost da je ulazni uzorak stvaran (iz skupa za treniranje).

    
<center>
<div class="fig figcenter fighighlight">
  <img src="https://dlunizg.github.io/assets/lab4/D.svg" width="30%">
</div>
</center>
    

Gore opisani ciljevi diskriminatora i generatora mogu se formalno izraziti u sljedećoj funkciji cilja:

$\min_G \max_D V(D,G) = E_{ \mathbf x \sim p_{data}(\mathbf x) } [\log D( \mathbf x)] + E_{ \mathbf z  \sim p_{\mathbf z}(\mathbf z) } [\log(1 - D(G( \mathbf z)))]$

Prvi pribrojnik predstavlja očekivanje procjene log vjerojatnosti da su uzorci iz skupa za treniranje stvarni. Drugi pribrojnik predstavlja očekivanje procjene log vjerojatnosti da generirani uzorci nisu stvarni, tj. da su umjetni. Diskriminator ima za cilj maksimizirati oba pribrojnika, dok generator ima za cilj minimizirati drugi pribrojnik. Svaki pribrojnik funkcije cilja može se jednostavno procijeniti za jednu mini grupu te se može procijeniti gradijent s obzirom na prametre obiju mreža. 

Treniranje dviju mreža (G i D) može se provesti istovremeno ili se u jednoj iteraciji prvo može trenirati jedna mreža a zatim druga. Dodatno, neki autori preporučuju da se u nekoliko uzastopnih iteracija trenira jedna mreža, a nakon toga druga mreža samo jednu iteraciju.

    
<center>
<div class="fig figcenter fighighlight">
  <img src="https://dlunizg.github.io/assets/lab4/GAN2.svg" width="50%">
</div>
</center>


Kod generiranja slika uspješnim se pokazao Deep Convolutional GAN (DCGAN) koji u skrivenim slojevima obiju mreža koristi konvolucijske slojeve. Za razliku od klasičnih konvolucijskih mreža, ovdje se ne koriste pooling slojevi nego se uzorkovanje provodi pomoću konvolucijskih slojeva koji imaju posmak veći od 1. Autori mreže preporučuju korištenje Batch normalizacije u svim slojevima osim u izlaznom sloju generatora te ulaznom i izlaznom sloju diskriminatora. Korištenje Leaky ReLU aktivacijskih funkcija u svim slojevima osim u izlaznim je još jedna specifičnost DCGAN-a kao i eliminacija potpuno povezanih slojeva.

    
<center>
<div class="fig figcenter fighighlight">
  <img src="https://dlunizg.github.io/assets/lab4/DCGAN.svg" width="50%">
</div>
</center>

### U sljedećih nekoliko blokova koda nalaze se inicijalizacijske postavke i gotove pomoćne funkcije

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from torchvision import datasets, transforms
import tqdm
from torchvision.utils import make_grid

import torch.distributions as tdist

import numpy as np
import tqdm

import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
def prepare_data_loaders(batch_size=32):
    train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./files', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                    torchvision.transforms.Resize(64),
                                    torchvision.transforms.ToTensor()
                               ])), batch_size=batch_size)

    test_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('./files', train=False, download=True,
                                   transform=torchvision.transforms.Compose([
                                       torchvision.transforms.Resize(64),
                                       torchvision.transforms.ToTensor()
                                   ])), batch_size=batch_size)
    
    return train_loader, test_loader

### Zadatak

Implementirajte DCGAN s generatorom i diskriminatorom. Arhitekura treba biti:
    
* Generator
    * Sloj 1 - Broj izlaznih kanala = 512, veličina jezgre = 4, veličina koraka = 1
    * Sloj 2 - Broj izlaznih kanala = 256, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 3 - Broj izlaznih kanala = 128, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 4 - Broj izlaznih kanala = 64, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 5 - Broj izlaznih kanala = 1, veličina jezgre = 4, veličina koraka = 2, padding = 1

* Diskriminator
    * Sloj 1 - Broj izlaznih konvolucija = 64, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 2 - Broj izlaznih konvolucija = 128, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 3 - Broj izlaznih konvolucija = 256, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 4 - Broj izlaznih konvolucija = 512, veličina jezgre = 4, veličina koraka = 2, padding = 1
    * Sloj 5 - Broj izlaznih konvolucija = 1, veličina jezgre = 4, veličina koraka = 1, padding = 0

Ulaz u generator $\mathbf z$ neka ima 100 elemenata prema normalnoj distribuciji $N(0,1)$. Ulazni podaci neka su MNIST brojevi skalirani na veličinu 64x64 te treniranje provedite kroz barem 20 epoha. U jednoj iteraciji provedite jednu optimizaciju generatora i jednu optimizaciju diskriminatora s po jednom mini grupom. Koristite tanh aktivacijsku funkciju za izlaz generatora i sigmoid aktivaciju za izlaz diskriminator, a za ostaje slojeve "propustljivi" ReLU sa "negative_slope" parametrom od 0.2. Batch normalizacija (jedan od podzadataka) ide iza svakog sloja.

**Podzadaci:**

 1. Vizualizirajte rezultate generiranja 100 novih uzoraka iz slučajnih vektora $\mathbf z$.
 2. Spremite težine istreniranog modela u datoteku "zad_gan.th" i uploadajte tu datoteku na Moodle.
 3. Na Moodle predajte vizualizaciju 1. podzadatka.
 4. Odgovorite na sljedeća pitanja **u bilježnici**. Bilježnicu na kraju predajte na Moodle.


Koristite sljedeći predložak:

**NAPOMENA**: Osim nadopunjavanja koda koji nedostaje, predložak se treba prilagođavati prema potrebi, a može i prema vlastitim preferencijama. Stoga **budite oprezni s tvrdnjama da vam neki dio koda ne radi!**

**Pitanje:**
    
U jednoj iteraciji provedite treniranje diskriminatora s dvije minigrupe, a generatora s jednom minigrupom. Ponovite isti postupak samo zamijenite mjesta generatora i diskriminatora. Vizualizirajte generirane uzorke i komentirajte rezultate.

**Odgovor:**

**Pitanje:**

Isključite batch normalizaciju u obje mreže. Komentirajte rezultate.

**Odgovor:**

In [11]:
'''
Generator koristi transponirane konvolucije (ConvTranspose2d) za povećanje dimenzionalnosti 
(upsampling) iz latentnog vektora do slike dimenzija 64x64. 
Batch normalizacija se koristi nakon svakog sloja osim izlaznog.
'''

class Generator(nn.Module):
    def __init__(self, latent_size=100):
        super(Generator, self).__init__()
        self.latent_size = latent_size

        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.tanh = nn.Tanh()
        
        # Sloj 1: Input (N, 100, 1, 1) -> Output (N, 512, 4, 4)
        self.ct1 = nn.ConvTranspose2d(latent_size, 512, 4, 1, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(512)
        
        # Sloj 2: Input (N, 512, 4, 4) -> Output (N, 256, 8, 8)
        self.ct2 = nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(256)
        
        # Sloj 3: Input (N, 256, 8, 8) -> Output (N, 128, 16, 16)
        self.ct3 = nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Sloj 4: Input (N, 128, 16, 16) -> Output (N, 64, 32, 32)
        self.ct4 = nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(64)
        
        # Sloj 5: Input (N, 64, 32, 32) -> Output (N, 1, 64, 64)
        self.ct5 = nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False)

    def forward(self, x):
        # Propagacija kroz slojeve: ConvT -> BatchNorm -> Activation
        x = self.ct1(x)
        x = self.bn1(x)
        x = self.lrelu(x)
        
        x = self.ct2(x)
        x = self.bn2(x)
        x = self.lrelu(x)
        
        x = self.ct3(x)
        x = self.bn3(x)
        x = self.lrelu(x)
        
        x = self.ct4(x)
        x = self.bn4(x)
        x = self.lrelu(x)
        
        x = self.ct5(x)
        out = self.tanh(x) # Izlazni sloj koristi Tanh
        
        return out

In [12]:
### Generator BEZ Batch Normalizacije
class GeneratorNoBN(nn.Module):
    def __init__(self, latent_size=100):
        super(GeneratorNoBN, self).__init__()
        self.latent_size = latent_size

        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.tanh = nn.Tanh()
        
        # Sloj 1: Input (N, 100, 1, 1) -> Output (N, 512, 4, 4)
        self.ct1 = nn.ConvTranspose2d(latent_size, 512, 4, 1, 0, bias=True) # Bias je True jer nema BN
        
        # Sloj 2: Input (N, 512, 4, 4) -> Output (N, 256, 8, 8)
        self.ct2 = nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=True)
        
        # Sloj 3: Input (N, 256, 8, 8) -> Output (N, 128, 16, 16)
        self.ct3 = nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=True)
        
        # Sloj 4: Input (N, 128, 16, 16) -> Output (N, 64, 32, 32)
        self.ct4 = nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=True)
        
        # Sloj 5: Input (N, 64, 32, 32) -> Output (N, 1, 64, 64)
        self.ct5 = nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=True)

    def forward(self, x):
        # Nema BN: ConvT -> Activation
        x = self.lrelu(self.ct1(x))
        x = self.lrelu(self.ct2(x))
        x = self.lrelu(self.ct3(x))
        x = self.lrelu(self.ct4(x))
        
        out = self.tanh(self.ct5(x))
        
        return out

In [13]:
'''
Diskriminator koristi standardne konvolucije (Conv2d) za smanjenje dimenzionalnosti 
(downsampling) i klasifikaciju slike kao stvarne ili lažne. 
Batch normalizacija se ne koristi na ulaznom sloju diskriminatora ni na izlaznom sloju.
'''

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.sigmoid = nn.Sigmoid()
        
        # Sloj 1: Input (N, 1, 64, 64) -> Output (N, 64, 32, 32)
        # Nema Batch Normalizacije na ulazu diskriminatora
        self.cv1 = nn.Conv2d(1, 64, 4, 2, 1, bias=False)
        
        # Sloj 2: Input (N, 64, 32, 32) -> Output (N, 128, 16, 16)
        self.cv2 = nn.Conv2d(64, 128, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        
        # Sloj 3: Input (N, 128, 16, 16) -> Output (N, 256, 8, 8)
        self.cv3 = nn.Conv2d(128, 256, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        
        # Sloj 4: Input (N, 256, 8, 8) -> Output (N, 512, 4, 4)
        self.cv4 = nn.Conv2d(256, 512, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(512)
        
        # Sloj 5: Input (N, 512, 4, 4) -> Output (N, 1, 1, 1)
        # Nema Batch Normalizacije na izlazu, koristi se Sigmoid
        self.cv5 = nn.Conv2d(512, 1, 4, 1, 0, bias=False)

    def forward(self, x):
        x = self.cv1(x)
        x = self.lrelu(x)
        
        x = self.cv2(x)
        x = self.bn2(x)
        x = self.lrelu(x)
        
        x = self.cv3(x)
        x = self.bn3(x)
        x = self.lrelu(x)
        
        x = self.cv4(x)
        x = self.bn4(x)
        x = self.lrelu(x)
        
        x = self.cv5(x)
        out = self.sigmoid(x)
        
        return out.squeeze() # Uklanja dimenzije veličine 1 (N, 1, 1, 1) -> (N)

In [14]:
### Diskriminator BEZ Batch Normalizacije
class DiscriminatorNoBN(nn.Module):
    def __init__(self):
        super(DiscriminatorNoBN, self).__init__()

        self.lrelu = nn.LeakyReLU(0.2, inplace=True)
        self.sigmoid = nn.Sigmoid()
        
        # Sloj 1: (N, 1, 64, 64) -> (N, 64, 32, 32). Prvi sloj nema BN.
        self.cv1 = nn.Conv2d(1, 64, 4, 2, 1, bias=True)
        
        # Sloj 2: (N, 64, 32, 32) -> (N, 128, 16, 16)
        self.cv2 = nn.Conv2d(64, 128, 4, 2, 1, bias=True)
        
        # Sloj 3: (N, 128, 16, 16) -> (N, 256, 8, 8)
        self.cv3 = nn.Conv2d(128, 256, 4, 2, 1, bias=True)
        
        # Sloj 4: (N, 256, 8, 8) -> (N, 512, 4, 4)
        self.cv4 = nn.Conv2d(256, 512, 4, 2, 1, bias=True)
        
        # Sloj 5: (N, 512, 4, 4) -> (N, 1, 1, 1). Izlazni sloj nema BN.
        self.cv5 = nn.Conv2d(512, 1, 4, 1, 0, bias=True)

    def forward(self, x):
        x = self.lrelu(self.cv1(x))
        x = self.lrelu(self.cv2(x))
        x = self.lrelu(self.cv3(x))
        x = self.lrelu(self.cv4(x))
        
        out = self.sigmoid(self.cv5(x))
        
        return out.squeeze()

In [15]:
def train(gmodel: Generator, dmodel: Discriminator, n_epochs=10, log_epochs=1, batch_size=32, learning_rate=1e-3, device='cpu',
          d_updates=1, g_updates=1):
    
    train_loader, test_loader = prepare_data_loaders(batch_size=batch_size)
    
    gmodel = gmodel.to(device)
    dmodel = dmodel.to(device)
    
    gmodel.train()
    dmodel.train()
    
    criterion = nn.BCELoss()
    
    g_optim = optim.Adam(gmodel.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    d_optim = optim.Adam(dmodel.parameters(), lr=learning_rate, betas=(0.5, 0.999))
    
    for epoch_idx in range(0, n_epochs):
        
        g_loss_accum, d_loss_accum = 0, 0
        
        for image_data, _ in tqdm.tqdm(train_loader):

            current_batch_size = image_data.shape[0] # Stvarna veličina batcha (zadnji batch može biti manji)
            image_data = image_data.to(device)

            # ------------------------------------
            #  1. Train Discriminator (D_updates puta)
            # ------------------------------------
            for _ in range(d_updates):
                dmodel.zero_grad()
                
                # --- Real data pass ---
                labels_real = torch.ones(current_batch_size, device=device).float()
                d_output_real = dmodel(image_data)
                d_err_real = criterion(d_output_real, labels_real)
                d_err_real.backward()

                # --- Fake data pass ---
                # Novi noise za svaki D update
                noise = torch.randn(current_batch_size, 100, 1, 1, device=device) 
                fake_image_data = gmodel(noise)
                
                labels_fake = torch.zeros(current_batch_size, device=device).float()
                
                # .detach() kako ne bismo računali gradijente za generator u ovom koraku
                d_output_fake = dmodel(fake_image_data.detach())
                d_error_fake = criterion(d_output_fake, labels_fake)
                d_error_fake.backward()
                
                d_optim.step()
                
                # Akumulacija gubitka samo za logiranje
                d_loss_accum += (d_err_real.item() + d_error_fake.item()) / d_updates


            # ------------------------------------
            #  2. Train Generator (G_updates puta)
            # ------------------------------------
            for _ in range(g_updates):
                gmodel.zero_grad()
                
                # Generiranje novog noise vektora
                noise = torch.randn(current_batch_size, 100, 1, 1, device=device)
                fake_image_data = gmodel(noise)

                # Želimo da D misli da su slike PRAVE (label=1)
                labels_g = torch.ones(current_batch_size, device=device).float()
                d_output_g = dmodel(fake_image_data) 
                
                g_error = criterion(d_output_g, labels_g)
                g_error.backward()
                
                g_optim.step()
                
                # Akumulacija gubitka
                g_loss_accum += g_error.item() / g_updates
            
        # Normalizacija gubitka po broju batcheva
        d_loss_avg = d_loss_accum / len(train_loader)
        g_loss_avg = g_loss_accum / len(train_loader)
            
        if (epoch_idx + 1) % log_epochs == 0:
            print(f"[{epoch_idx+1}/{n_epochs} - D:{d_updates}x / G:{g_updates}x]: d_loss = {d_loss_avg:.5f} g_loss = {g_loss_avg:.5f}")
            
    gmodel.eval()
    dmodel.eval()
    
    return gmodel, dmodel

In [16]:
gmodel_d1g1 = Generator()
dmodel_d1g1 = Discriminator()

print("--- Treniranje: D:1x, G:1x (default)---")
gmodel_d1g1, dmodel_d1g1 = train(gmodel_d1g1, dmodel_d1g1, n_epochs=20, batch_size=128, device='cpu', d_updates=1, g_updates=1)

random_sample_d1g1 = gmodel_d1g1(torch.randn(100, 100, 1, 1).to('cpu')).view(100, 64, 64).data.cpu().numpy()
plt.figure(figsize=(10, 10))
for idx in range(100):
    plt.subplot(10, 10, idx+1)
    plt.imshow(random_sample_d1g1[idx, ...], cmap='gray')
    plt.axis('off')
plt.suptitle('Rezultati: Diskriminator 1x, Generator 1x', fontsize=16)
plt.show()

--- Treniranje: D:1x, G:1x (default)---


100%|██████████| 469/469 [23:24<00:00,  2.99s/it]


[1/20 - D:1x / G:1x]: d_loss = 0.68125 g_loss = 4.51061


100%|██████████| 469/469 [22:56<00:00,  2.93s/it]


[2/20 - D:1x / G:1x]: d_loss = 0.32717 g_loss = 5.52453


...

[18/20 - D:1x / G:1x]: d_loss = 0.00000 g_loss = 16.47643


100%|██████████| 469/469 [20:26<00:00,  2.62s/it]


[19/20 - D:1x / G:1x]: d_loss = 0.00000 g_loss = 16.00640


 49%|████▉     | 230/469 [09:58<10:22,  2.60s/it]


KeyboardInterrupt: 

In [ ]:
with open('zad_gan.th', 'wb') as f:
    torch.save(gmodel_d1g1.state_dict(), f)

In [ ]:
gmodel_d2g1 = Generator()
dmodel_d2g1 = Discriminator()

# Ovo je scenarij u kojem D ima više prilika za učenje
print("--- Treniranje: D:2x, G:1x ---")
gmodel_d2g1, dmodel_d2g1 = train(gmodel_d2g1, dmodel_d2g1, n_epochs=20, batch_size=128, device='cpu', d_updates=2, g_updates=1)

random_sample_d2g1 = gmodel_d2g1(torch.randn(100, 100, 1, 1).to('cpu')).view(100, 64, 64).data.cpu().numpy()
plt.figure(figsize=(10, 10))
for idx in range(100):
    plt.subplot(10, 10, idx+1)
    plt.imshow(random_sample_d2g1[idx, ...], cmap='gray')
    plt.axis('off')
plt.suptitle('Rezultati: Diskriminator 2x, Generator 1x', fontsize=16)
plt.show()

In [ ]:
gmodel_d1g2 = Generator()
dmodel_d1g2 = Discriminator()

# Ovo je scenarij u kojem G ima više prilika za učenje (očekuje se mode collapse)
print("--- Treniranje: D:1x, G:2x ---")
gmodel_d1g2, dmodel_d1g2 = train(gmodel_d1g2, dmodel_d1g2, n_epochs=20, batch_size=128, device='cpu', d_updates=1, g_updates=2)

random_sample_d1g2 = gmodel_d1g2(torch.randn(100, 100, 1, 1).to('cpu')).view(100, 64, 64).data.cpu().numpy()
plt.figure(figsize=(10, 10))
for idx in range(100):
    plt.subplot(10, 10, idx+1)
    plt.imshow(random_sample_d1g2[idx, ...], cmap='gray')
    plt.axis('off')
plt.suptitle('Rezultati: Diskriminator 1x, Generator 2x', fontsize=16)
plt.show()

In [ ]:
gmodel_nobn = GeneratorNoBN()
dmodel_nobn = DiscriminatorNoBN()

print("--- Treniranje: BEZ Batch Normalizacije ---")
# Omjer 1:1
gmodel_nobn, dmodel_nobn = train(gmodel_nobn, dmodel_nobn, n_epochs=20, batch_size=128, device='cpu', d_updates=1, g_updates=1)

random_sample_nobn = gmodel_nobn(torch.randn(100, 100, 1, 1).to('cpu')).view(100, 64, 64).data.cpu().numpy()
plt.figure(figsize=(10, 10))
for idx in range(100):
    plt.subplot(10, 10, idx+1)
    plt.imshow(random_sample_nobn[idx, ...], cmap='gray')
    plt.axis('off')
plt.suptitle('Rezultati: BEZ Batch Normalizacije', fontsize=16)
plt.show()